In [44]:
import numpy as np
from tkinter import *
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
# 애니메이션 을 활용할 모듈 임포트
from matplotlib.animation import FuncAnimation
import math

In [45]:
# 토끼 의 그래프를 그리기 위한 차트 준비
fig = Figure(figsize=(5,3), dpi=100)
ax = fig.add_subplot(111)
ax.set_xlim(0, 11)
ax.set_ylim(0, 11)
# h_xdata, h_ydata : 토끼의 구간별 거리 데이터
# x_list, y_list : 토끼 그래프 주변에 회귀선을 찾기 위한 가상의 그래프가 
# 그려질 데이터
h_xdata, h_ydata, x_list, y_list = [], [], [], []

In [46]:
grad_fig = Figure( figsize=(5,3), dpi=100)
grad_ax=grad_fig.add_subplot(111)
grad_ax.set_xlim(-0.1, 1.1)
grad_ax.set_ylim(0, 100)
t_xdata, t_ydata = [], []

In [47]:
def update():
    # 토끼의 속도를 표시하고 있는 스핀 박스 값 추출
    h_a = float(h_aSpbox.get())
    # 0부터 10까지 11개의 타임데이터(x 축데이터 )생성후 그 수 만큼 반복실행
    for t in np.linspace(0, 10, 11):
        # 시간에 속도를 곱해서 y 축데이터 계산 & x , y 축데이터를 리스트에 저장
        h_y = h_a * t
        h_xdata.append(t)
        h_ydata.append(h_y)
    ax.set_xlabel('Time(hour)')
    ax.set_ylabel('Distance(km)')
    ax.set_title('Linear Regression')
    ax.plot(h_xdata,h_ydata, 'ro', label='Rabbit')
    ax.legend()
    fig.canvas.draw()

In [48]:
def showLines():
    # 스핀 박스 값들을 추출
    h_a = float(h_aSpbox.get()) # 토끼의 속도 
    h_s = float(h_sSpbox.get()) # 회귀선의 간격
    # 속도(1) + 회귀선간격(0.1) * 5  를 저장합니다 
    a_val = h_a + (h_s * 5)  
    # 저장값 1.5     1.5부터 0.5 까지 0.1 씩줄이며 반복하기 위한 준비  
    h_xdata = []
    h_ydata = []
    # x 좌표값으로 반복 실행
    for i in np.linspace(0, 10, 11):
        a = a_val - (i*h_s)  # a_val 값 1.5 부터 0.1 씩 뺀값 1.5~0.5 값 발생
        # 계산된 a값(기울기)로 10번의 연산을 하여 10개으이 x 좌표값과 y좌표값을 계산
        for t in np.linspace(0,10,11):
            # 계산된 좌표값들을 리스트에 저장
            h_y = a*t
            h_xdata.append(t)
            h_ydata.append(h_y)
        # x 좌표값과 y 좌표값으로 차트 드로잉
        ax.plot(h_xdata,h_ydata, alpha=0.2)
    fig.canvas.draw()

In [49]:
# 애니메이션 적용을 위한 차트의 최초 설정 
# 이름 부여(ln, dn, : 튜플 형식의 변수)
ln, = grad_ax.plot(0, 0)  # 라인차트
dn, = grad_ax.plot([], [], 'ro')  # 스캐터 차트
# 위 동작에서 차트가 그려지지는 않습니다. 이는 차트의 정류와 형식을 초기화하는 동작
# 차트가 1회에 한번씩 덧그려지면서 현재 데이터의 표시동작을 반복하며, 
# 애니메이션 효과를 낼 예정입니다

In [50]:
# 애니메이션 차트 초기화 함수 -   animate 함수의 일부요소로 사용됩니다
def init():
    grad_ax.set_xlim(-0.1, 1.1)
    grad_ax.set_ylim(0, 100)
    return dn, ln,    

In [51]:
# 기준속도(기울기 : 1)과  다른 기울기(매개변수로 전달) 간의 코스트값 계산 함수
def get_cost(a_val):
    h_a = float(h_aSpbox.get()) # 윈도우에 설정된 속도(기울기) 추출
    cost = 0
    for i in range(0, 11, 1):
        cost += pow( (a_val*i - h_a*i), 2)
    return cost

In [52]:
# 애니메이션 동작에 관련하고 있는 함수 (dn 을 이용하여 점(scatter)을 찍는 동작)
def animateFrame(frame):
    h_a = float(h_aSpbox.get()) 
    h_s = float(h_sSpbox.get())
    a_val = h_a + (h_s * 5)  # 최고 속도(프레임 시작값)
    i = frame * h_s   # 각 프레임의 x 좌표값
    a = a_val - i  # 각 프레임의 속도
    # 최고 속도에서 각 프레임별로 -0.1 씩 계산하며 프레임별 속도 생성
    t_xdata.append(i)  # x 좌표값 저장
    t_ydata.append(get_cost(a))  # 프레임별 속도를 전달한 함수로 코스트값 계산 저장
    dn.set_data(t_xdata, t_ydata)  # 좌표값으로 애니메이션 설정(dn 사용)
    return dn, ln,   # 준비된 애니메이션 차트 리턴

In [53]:
def gradient():
    # grad_fig : 그려질 차트 이름(위에서 이미 생성완료)
    # animateFrame : 애니매이션 동작이 들어있는 함수이름
    # frames=np.linspace(0, 10, 11) : 애니메이션 프레임의 번호(갯수결정)
    # init_func=init : 애니메이션 시작전 초기화 함수 설정
    ani = FuncAnimation(grad_fig, animateFrame, frames=np.linspace(0, 10, 11), \
                       init_func=init)
    # 함수가 호출이 되면 animateFrame 함수에 frames 에 있는 숫자들이 전달인수로
    # 전달됩니다.(한번에 11개의 데이터가 한번 호출에 모두 전달되는 형식 이지만.....
    # 하나씩 전달되어 11번을 호출한다고 생각하면 이해에 도움이 됩니다.)
    
    # animateFrame 함수에 0~10 까지 11개의 숫자를 보내고, 11개의 차트를 리턴받아
    # grad_fig 에 그립니다. FuncAnimation  함수가 11개의 차트를 차례로 애니메이션처럼
    # 그립니다
    
    \grad_ax.set_title('Gradient descent')
    grad_ax.set_ylabel("Total Cost")
    grad_ax.set_xlabel("Variance")
    grad_fig.canvas.draw()

In [54]:
#main
main = Tk()
main.title("토끼의 선형회귀")
main.geometry()
label=Label(main, text='토끼의 선형회귀')
label.config(font=("굴림", 18))
label.grid(row=0,column=0,columnspan=4)
h_aVal = DoubleVar(value=1.0)
h_aSpbox = Spinbox(main, textvariable=h_aVal ,from_=0, to=10, increment=1)
h_aSpbox.config(state='readonly')
h_aSpbox.grid(row=1,column=2)
h_aLabel=Label(main, text='토끼의 시속 (km/h) : ')                
h_aLabel.grid(row=1,column=0,columnspan=2)
h_sVal  = DoubleVar(value=0.1)
h_sSpbox = Spinbox(main, textvariable=h_sVal ,from_=0, to=2, increment=0.01)
h_sSpbox.config(state='readonly')
h_sSpbox.grid(row=2,column=2)
h_sLabel=Label(main, text='회귀선 간격 (km/h) : ')                
h_sLabel.grid(row=2,column=0,columnspan=2)
btn1 = Button(main,text="Run",width=20,height=3,command=lambda:update())
btn1.grid(row=3, column=0)
btn2 = Button(main,text="Lines",width=20,height=3,command=lambda:showLines())
btn2.grid(row=3, column=1)
btn3 = Button(main,text="Gradient",width=20,height=3,command=lambda:gradient())
btn3.grid(row=3, column=2)

canvas = FigureCanvasTkAgg(fig, main)
canvas.get_tk_widget().grid(row=4,column=0,columnspan=4)

grad_canvas = FigureCanvasTkAgg(grad_fig, main)
grad_canvas.get_tk_widget().grid(row=5,column=0,columnspan=4) 

In [55]:
main.mainloop()